# Fiddle Basic API

*Please see https://github.com/google/fiddle/blob/main/docs/colabs.md for other colabs.*

This colab walks through the core Fiddle abstractions and how they work.

In [1]:
import copy
import dataclasses
from typing import List

!pip install fiddle


import fiddle as fdl

## `fdl.Config`

The most common API for Fiddle is the `fdl.Config` type. `fdl.Config`s
correspond to a call to a function or class. (For classes, obviously, calling
the class creates a class instance.)

In [2]:
@dataclasses.dataclass
class Foo:
  a: int


config = fdl.Config(Foo, a=2)
fdl.build(config)

Foo(a=2)

`fdl.Config` stores two key things:

*   A reference to the function or class to call
*   Arguments to call it with

The key affordance of Fiddle is that the arguments are mutable, and can
themselves contain sub-configuration. Let's first look at the mutability:

In [3]:
config.a = 3
fdl.build(config)

Foo(a=3)

These arguments can be retrieved as well. For example, you can double a value in
an existing configuration like so,

In [4]:
config.a = 2 * config.a
fdl.build(config)

Foo(a=6)

### Nested configuration

Let's look at the case when arguments themselves contain sub-configs,

In [5]:
@dataclasses.dataclass
class Bar:
  x: Foo
  y: List[Foo]


foo_config = fdl.Config(Foo, a=1)
config = fdl.Config(Bar, foo_config, [foo_config])
bar = fdl.build(config)
bar

Bar(x=Foo(a=1), y=[Foo(a=1)])

One key thing that Fiddle provides is the ability to have shared instances. Each
`fdl.Config` object corresponds to a single call during `build`. So, in the
above case, the `Foo` objects are actually the same,

In [6]:
bar.x is bar.y[0]

True

If you don't want these to be the same instance, you can copy `foo_config`;
please see the "Copying configuration" section below.

### Setting and retrieving parameters

Setting an unknown attribute will raise an error,

In [7]:
try:
  foo_config.qux = 4
except TypeError as e:
  %html <span style="color: red">TypeError: {e}</span>
else:
  raise AssertionError("This should raise an error!")

Configuring `**kwargs` arguments are supported,

In [8]:
def args_and_kwargs(a, *args, **kwargs):
  return a, args, kwargs


config = fdl.Config(args_and_kwargs, a=4, b=12)
fdl.build(config)

(4, (), {'b': 12})

but `*args` are currently unsupported,

In [9]:
try:
  fdl.Config(args_and_kwargs, 4, 7)
except NotImplementedError as e:
  %html <span style="color: red">NotImplementedError: {e}</span>
else:
  raise AssertionError("This should raise an error!")

### Type annotations

The `fdl.Config` type can take a type parameter, which indicates the output type
of the function or class being configured.

In [10]:
foo_config: fdl.Config[Foo] = fdl.Config(Foo, 1)

A call to `fdl.build(foo_config)` correctly hints its output type as `Foo`,
which helps static type checkers.

In the future, we might check argument types, but this is not currently
available. (Unfortunately, standard Python type tooling is insufficiently
powerful to express the types precisely.) So you can currently write
`fdl.Config(Foo, "string_not_int")`.

## `fdl.Partial`

Let's say you have a checkpointer which takes the current train state and number
of steps. This might not be exactly the API you would create if you intended to
use Fiddle from the beginning, but Fiddle is designed to be able to configure
existing classes.

In [11]:
from typing import Any
import dataclasses


@dataclasses.dataclass
class Checkpointer:
  train_state: Any
  num_steps: int
  base_directory: str
  checkpoints_to_keep: int

  def clear_old_checkpoints(self):
    print("  Clearing checkpoints to limit to", self.checkpoints_to_keep,
          "checkpoints")

  def save(self):
    self.clear_old_checkpoints()
    print("  Saved to", self.base_directory, "at num_steps =", self.num_steps)


@dataclasses.dataclass
class Trainer:
  checkpointer_cls: Any
  num_steps: int = dataclasses.field(default=0)

  def train_epoch(self, epoch_steps: int):
    print("Training for one epoch ...")
    train_state = None
    self.num_steps += epoch_steps
    self.checkpointer_cls(
        train_state=train_state, num_steps=self.num_steps).save()

Then let's say you want to configure the `base_directory` and
`checkpoints_to_keep` parameters, and let the `train_state` and `num_steps`
parameters be set by caller code at runtime. Then we can create a `fdl.Partial`
object to represent the `checkpointer_cls`,

In [12]:
def trainer_config() -> fdl.Config:
  checkpointer_config = fdl.Partial(
      Checkpointer, base_directory="/path/to/my/dir", checkpoints_to_keep=5)
  return fdl.Config(Trainer, checkpointer_cls=checkpointer_config)


config = trainer_config()

# Experimental overrides can be applied as normal.
config.checkpointer_cls.checkpoints_to_keep = 10

# The trainer can be built, and we can mock-train two epochs.
trainer = fdl.build(config)
trainer.train_epoch(10)
trainer.train_epoch(20)

Training for one epoch ...
  Clearing checkpoints to limit to 10 checkpoints
  Saved to /path/to/my/dir at num_steps = 10
Training for one epoch ...
  Clearing checkpoints to limit to 10 checkpoints
  Saved to /path/to/my/dir at num_steps = 30


When built, the `checkpointer_cls` becomes a `functools.partial` instance, partially specifying the arguments for `Checkpointer`:

In [13]:
fdl.build(config.checkpointer_cls)

functools.partial(<class '__main__.Checkpointer'>, base_directory='/path/to/my/dir', checkpoints_to_keep=10)

`fdl.Partial` can also be called on functions:

In [14]:
def foo(a, b):
  return a + b


foo_partial = fdl.build(fdl.Partial(foo, a=1))
foo_partial(b=2)

3

## Building configuration (`fdl.build`): anti-patterns to avoid

As you've seen before, `fdl.build` is the core function creating actual classes
from their configuration.

A key affordance is that when used properly, built objects have any
configuration "closured" in. So that if the configuration is ever mutated later,
then the built objects are not changed. To demonstrate:

In [ ]:
def foo(a):
  return a


config = fdl.Partial(foo, a=1)
foo_partial = fdl.build(config)
config.a = 4
foo_partial()

1

We strongly encourage `fdl.build` to be a single transition point from
configuration to built objects. Returning `fdl.Config` objects as the output of
a function being configured is discouraged. Further, calling `fdl.build` within
a `fdl.build` call will raise an error,

In [ ]:
def bar():
  sub_config = fdl.Config(foo, a=1)
  return fdl.build(sub_config)

try:
  fdl.build(fdl.Config(bar))
except fdl.BuildError as e:
  error_type = type(e.original_error).__name__
  %html <span style="color: red">{error_type}: {e.original_error}</span>
else:
  raise AssertionError("This should raise an error!")

As you see above, errors raised in the course of `fdl.build` are wrapped in a
`BuildError`. This currently provides one "path" from the root config to the
object which raised the error, which helps debug configurations.

In [ ]:
def my_error_function() -> Foo:
  raise ValueError("Error!")

foo_config = fdl.Config(Foo, a=1)
config = fdl.Config(Bar, foo_config, [foo_config, fdl.Config(my_error_function)])
try:
  fdl.build(config)
except fdl.BuildError as e:
  %html <span style="color: red">BuildError: {e}</span>
else:
  raise AssertionError("This should raise an error!")

## Copying configuration

`fdl.Config` and `fdl.Partial` objects can be shallow-copied by just calling
`copy.copy`, or `fdl.Config`/`fdl.Partial` with an existing config,

In [ ]:
def foo(a, b):
  return a, b


config = fdl.Config(foo, a="a", b="original_b")
config2 = copy.copy(config)  # Alternate: fdl.Config(config)
config2.b = "new_b"
print("Original:", fdl.build(config))
print("Shallow copy:", fdl.build(config2))

Original: ('a', 'original_b')
Shallow copy: ('a', 'new_b')


Deep copies are also supported, with the expected semantic that shared objects
are copied, but still shared.

In [ ]:
@dataclasses.dataclass
class FooWithId(Foo):

  def __repr__(self):
    return f"Foo<{id(self)}>(a={self.a})"


foo_config = fdl.Config(FooWithId, a=1)
config = fdl.Config(Bar, foo_config, [foo_config])
config2 = copy.copy(config)
config3 = copy.deepcopy(config)

# Change the original sub-object.
foo_config.a = 4

print("Original:", fdl.build(config))

# This reflects the post-copy change to `foo_config.a`.
print("Shallow copy:", fdl.build(config2))

# This has the original value for `foo_config.a`.
print("Deep copy:", fdl.build(config3))

Original: Bar(x=Foo<140115004801232>(a=4), y=[Foo<140115004801232>(a=4)])
Shallow copy: Bar(x=Foo<140114977767632>(a=4), y=[Foo<140114977767632>(a=4)])
Deep copy: Bar(x=Foo<140114977767824>(a=1), y=[Foo<140114977767824>(a=1)])


In general, we advise users to use `copy.deepcopy`, unless they intend to share
sub-configs with the original object.

## Serialization, equality, and hashing

We'll introduce serialization in more depth later, but please note that
`fdl.Config` objects are pickleable. Also, an `__eq__` operator for `Config`
works as expected. But we mostly suggest using `__eq__` in unit tests, since it
is plausible that two configurations may produce the same object, but have
different expressions (e.g. `fdl.Config(lambda x: x, 1)` can be used in place of
`1`).

In [ ]:
import pickle

reloaded = pickle.loads(pickle.dumps(config))
assert reloaded is not config, "Reloaded object should not be identical"
assert reloaded == config, "Reloaded object should be the same by __eq__"

Since `fdl.Config` objects are mutable, they are not hashable. If you would like
to put them in a dictionary, please hash their IDs.

In [ ]:
try:
  hash(config)
except TypeError as e:
  %html <span style="color: red">TypeError: {e}</span>
else:
  raise AssertionError("This should raise an error!")

## Internal details

Please do not access private fields of `fdl.Config` objects, but for building
your mental model, you can think of these objects as maintaining a reference to
the class/function being configured, and their arguments,

In [ ]:
print("Function configured:", config.__fn_or_cls__)
print("Arguments:", config.__arguments__)

Function configured: <class '__main__.Bar'>
Arguments: {'x': <Config[FooWithId(a=4)]>, 'y': [<Config[FooWithId(a=4)]>]}


As such, Fiddle objects have been designed to behave like one would expect
Python objects to behave. For example, although we discourage it, you could
configure a function created in a closure,

In [ ]:
def make_configured(x):

  def inner(y):
    return x + y

  return fdl.Config(inner, y=1)


config1 = make_configured(3)
config2 = make_configured(4)

assert fdl.build(config1) == 3 + 1
assert fdl.build(config2) == 4 + 1

However, using Fiddle in such ways will likely break serialization, and will be
more difficult to debug.